## Agile Hardware Design
***
# Encapsulation

<img src="../images/logo.svg" alt="agile hardware design logo" style="float:right"/>

## Based on class note from Prof. Scott Beamer
## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* Scala methods, recursion, and objects
* Using Scala methods to build Chisel components
* Chisel Bundles

## Loading The Chisel Library Into a Notebook

In [25]:
//interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
// Before we start. We test for UCB stcture and make sure they worked.
// Below, we test for UCB stcture and make sure they worked.
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
println("path: "+path)

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/006_Encap/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/006_Encap/001_Code/source/load-ivy.sc"

In [26]:
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [27]:
//import chisel3._
//import chisel3.util._
//import chiseltest._
//import chiseltest.RawTester.test
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

In [28]:
// Test
class RegLand extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val en  = Input(Bool())
        val out = Output(Bool())
    })
    val r = Reg(Bool())
//    val r = RegInit(0.B)
    r := io.in
    io.out := r
//     io.out := RegNext(io.in, 0.B)
//     io.out := RegEnable(io.in, 0.B, io.en)
}
println (getVerilog(new RegLand))

Elaborating design...
Done elaborating.
module RegLand(
  input   clock,
  input   reset,
  input   io_in,
  input   io_en,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg  r; // @[cmd27.sc 7:16]
  assign io_out = r; // @[cmd27.sc 10:12]
  always @(posedge clock) begin
    r <= io_in; // @[cmd27.sc 9:7]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin 

defined class RegLand

In [29]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n > 0)
    val regs = Seq.fill(n)(Reg(Bool()))
    regs(0) := io.in
    for (i <- 1 until n)
        regs(i) := regs(i-1)
    io.out := regs(n-1)
}
//println(getVerilog(new DelayNCycles(2)))

defined class DelayNCycles

In [30]:
println(getVerilog(new DelayNCycles(2)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg  regs_0; // @[cmd28.sc 7:31]
  reg  regs_1; // @[cmd28.sc 7:31]
  assign io_out = regs_1; // @[cmd28.sc 11:12]
  always @(posedge clock) begin
    regs_0 <= io_in; // @[cmd28.sc 8:13]
    regs_1 <= regs_0; // @[cmd28.sc 10:17]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERI

## Scala Methods

* Syntax examples on right
  * Multi-line bodies need braces
  * Last line is returned
  * Normally inside `class` or `object`

* Arguments are immutable by default

* Can give default argument values


### One Line Method ###

In [31]:
def plusOne(n: Int) = n + 1

plusOne(5)

def plusX(n: Int, x: Int = 1) = n + x

plusX(5,2)
plusX(5)

defined function plusOne
res30_1: Int = 6
defined function plusX
res30_3: Int = 7
res30_4: Int = 6

### Several Lines Method ###

In [32]:
def plusOne1(n: Int) = {
    n + 1
}

def plusX1 (n: Int, x: Int = 1) = {
    n + x
}

plusOne1 (3)
plusX1 (5, 2)

defined function plusOne1
defined function plusX1
res31_2: Int = 4
res31_3: Int = 7

## Recursive Scala Methods


* Need be sure to specify return type
  * `Unit` is nothing (like void)
* Be sure to think of base case
* Helpful for iteration or decomposing a problem


In [33]:
// recusrive sum
def recSum(n: Int): Int = {
    if (n <= 0) 0
    else n + recSum(n - 1)
}

for (n <- 0 until 10)
   println("n = " + n + ", reSum(" + n + ") = " + recSum(n))

n = 0, reSum(0) = 0
n = 1, reSum(1) = 1
n = 2, reSum(2) = 3
n = 3, reSum(3) = 6
n = 4, reSum(4) = 10
n = 5, reSum(5) = 15
n = 6, reSum(6) = 21
n = 7, reSum(7) = 28
n = 8, reSum(8) = 36
n = 9, reSum(9) = 45


defined function recSum

In [34]:
// Fibobacci Recusrive Sequence
def fib(n: Int): Int = {
    if (n < 2) n
    else fib(n-1) + fib(n-2)
}

for (n <- 0 until 10)
    println("n = " + n + ", fib(" + n + ") = " + fib(n))

n = 0, fib(0) = 0
n = 1, fib(1) = 1
n = 2, fib(2) = 1
n = 3, fib(3) = 2
n = 4, fib(4) = 3
n = 5, fib(5) = 5
n = 6, fib(6) = 8
n = 7, fib(7) = 13
n = 8, fib(8) = 21
n = 9, fib(9) = 34


defined function fib

## Using Scala to Construct Chisel Components

* Chisel components are just objects in Scala, so can use methods to build them up
  * Can _encapsulate_ (hide) complexity
  * Can declare once, use in many places
  * Use recursion to perform iteration (to implement paramterized flexibility)

* Although testers requires a `Module`, valid to construct components outside
  * Chisel's `Module` & `Bundle` add things to class (through inheritance)

## DelayN (shift register) Revised with Helper Function

In [35]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    var lastConn = io.in
    for (i <- 0 until n)
        lastConn = RegNext(lastConn)
    io.out := lastConn
}
//printVerilog(new DelayNCycles(2))

defined class DelayNCycles

In [36]:
println(getVerilog(new DelayNCycles(2)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd34.sc 9:27]
  reg  lastConn; // @[cmd34.sc 9:27]
  assign io_out = lastConn; // @[cmd34.sc 10:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd34.sc 9:27]
    lastConn <= REG; // @[cmd34.sc 9:27]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATO

In [37]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    def helper(n: Int, lastConn: Bool): Bool = {
        if (n == 0) lastConn
        else helper(n-1, RegNext(lastConn))
    }
    io.out := helper(n, io.in)
}
//printVerilog(new DelayNCycles(2))
println(getVerilog(new DelayNCycles(2)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd36.sc 9:33]
  reg  REG_1; // @[cmd36.sc 9:33]
  assign io_out = REG_1; // @[cmd36.sc 11:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd36.sc 9:33]
    REG_1 <= REG; // @[cmd36.sc 9:33]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `

defined class DelayNCycles

## Scala `object`

* _Singleton_ object
  * Exacly one always exists
  * By contrast, must instantiate a `class` to use

* Typical uses
  * Shared state (constant or mutable)
  * Stateless functions
  * Factory methods (as _companion object_)
  * ChiselEnums

### Create Object and Access the Method of class MyPair ###

In [38]:
class MyPair(a: Int, b: Int) {
    def sum() = a + b
}

object MyPair {
    var numPairs = 0
    def apply(a: Int, b: Int) = {
        numPairs += 1
        new MyPair(a,b)
    }
    def apply(a: Int): MyPair = apply(a, 0)
}

println("val mpc = new MyPair(3,4):")
val mpc = new MyPair(3,4)
println("mpc.sum(): " + mpc.sum())
println("mpc.sum: " + mpc.sum)

val mpc = new MyPair(3,4):
mpc.sum(): 7
mpc.sum: 7


defined class MyPair
defined object MyPair
mpc: MyPair = ammonite.$sess.cmd37$Helper$MyPair@5d3d4c56

### Direct Access Method and Data with Object ###

In [39]:
class MyPair(a: Int, b: Int) {
    def sum() = a + b
}

object MyPair {
    var numPairs = 0
    def apply(a: Int, b: Int) = {
        numPairs += 1
        new MyPair(a,b)
    }
    def apply(a: Int): MyPair = apply(a, 0)
}

println ("MyPair(2,3): " + MyPair(2,3))
println ("MyPair(2,3).sum: " + MyPair(2,3).sum)
println ("MyPair.numPairs: " + MyPair.numPairs)
println()
println ("val mpo = MyPair(3):")
val mpo = MyPair(3)
println ("mpo.sum: " + mpo.sum)
println ("MyPair.numPairs: " + MyPair.numPairs)

MyPair(2,3): ammonite.$sess.cmd38$Helper$MyPair@78e87c11
MyPair(2,3).sum: 5
MyPair.numPairs: 2

val mpo = MyPair(3):
mpo.sum: 3
MyPair.numPairs: 3


defined class MyPair
defined object MyPair
mpo: MyPair = ammonite.$sess.cmd38$Helper$MyPair@6602c532

## Factory Method for `MyCounter` ##

In [40]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}

object MyCounter {
    def apply(maxVal: Int) = new MyCounter(maxVal)
}

defined class MyCounter
defined object MyCounter

In [41]:
//printVerilog(new MyCounter(15))
println(getVerilog(new MyCounter(15)))

Elaborating design...
Done elaborating.
module MyCounter(
  input        clock,
  input        reset,
  input        io_en,
  output [3:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [3:0] count; // @[cmd39.sc 6:24]
  wire [3:0] _T_2 = count + 4'h1; // @[cmd39.sc 9:28]
  assign io_out = count; // @[cmd39.sc 14:12]
  always @(posedge clock) begin
    if (reset) begin // @[cmd39.sc 6:24]
      count <= 4'h0; // @[cmd39.sc 6:24]
    end else if (io_en) begin // @[cmd39.sc 7:18]
      if (count < 4'hf) begin // @[cmd39.sc 8:33]
        count <= _T_2; // @[cmd39.sc 9:19]
      end else begin
        count <= 4'h0; // @[cmd39.sc 11:19]
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $rand

## MyCounter without `Module`

### Factory Method without Module ###
1. Define a class MyCounter without extends Module.
2. Define an object to access MyCounter object. 

class MyCounter(maxVal: Int, en: Bool) {
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
}

object MyCounter {
    def apply(maxVal: Int, en: Bool) = {
        val mc = new MyCounter(maxVal, en)
        mc.count
    }
}

In [42]:
class MyCounter(maxVal: Int, en: Bool) {
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
}

object MyCounter {
    def apply(maxVal: Int, en: Bool) = {
        val mc = new MyCounter(maxVal, en)
        mc.count
    }
}

defined class MyCounter
defined object MyCounter

### Create class to access the MyCounter Object ### 

In [43]:
class CounterInstMod(n: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt())
    })
    io.count := MyCounter(n, io.en)
}

defined class CounterInstMod

In [44]:
println(getVerilog(new CounterInstMod(4)))

Elaborating design...
Done elaborating.
module CounterInstMod(
  input        clock,
  input        reset,
  input        io_en,
  output [2:0] io_count
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [2:0] REG; // @[cmd41.sc 2:24]
  wire [2:0] _T_2 = REG + 3'h1; // @[cmd41.sc 5:28]
  assign io_count = REG; // @[cmd42.sc 6:14]
  always @(posedge clock) begin
    if (reset) begin // @[cmd41.sc 2:24]
      REG <= 3'h0; // @[cmd41.sc 2:24]
    end else if (io_en) begin // @[cmd41.sc 3:15]
      if (REG < 3'h4) begin // @[cmd41.sc 4:33]
        REG <= _T_2; // @[cmd41.sc 5:19]
      end else begin
        REG <= 3'h0; // @[cmd41.sc 7:19]
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`end

## Chisel Counter ###

1. In the previous slides, we write our own counter (MyCounter).
2. In this example, we can use Chisel Counter Library:
[`Counter`](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/Counter.html)
3. Or click the link below:
https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/Counter.html

In [45]:
class CounterInstMod(n: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt())
        val limit = Output(Bool())
    })
    val (value, wrap) = Counter(io.en, n)
//     val (value, wrap) = Counter(0 until n by 2, io.en)
    io.count := value
    io.limit := wrap
}

defined class CounterInstMod

In [46]:
//printVerilog(new CounterInstMod(4))
println(getVerilog(new CounterInstMod(4)))

Elaborating design...
Done elaborating.
module CounterInstMod(
  input        clock,
  input        reset,
  input        io_en,
  output [1:0] io_count,
  output       io_limit
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [1:0] value; // @[Counter.scala 60:40]
  wire  wrap_wrap = value == 2'h3; // @[Counter.scala 72:24]
  wire [1:0] _wrap_value_T_1 = value + 2'h1; // @[Counter.scala 76:24]
  assign io_count = value; // @[cmd44.sc 9:14]
  assign io_limit = io_en & wrap_wrap; // @[Counter.scala 118:17 Counter.scala 118:24]
  always @(posedge clock) begin
    if (reset) begin // @[Counter.scala 60:40]
      value <= 2'h0; // @[Counter.scala 60:40]
    end else if (io_en) begin // @[Counter.scala 118:17]
      value <= _wrap_value_T_1; // @[Counter.scala 76:15]
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INI

## Chisel `Bundle`

* Aggregate type with named fields
  * Like a `struct` in C
* Not only used to make IO interfaces
* Can declare once & use in many places

In [47]:
class Mag extends Bundle {
    val m = UInt(4.W)
}

class OutMod(a: Int) extends Module {
    val io = IO(Output(new Mag))
    io.m := a.U
}

//printVerilog(new OutMod(2))
println(getVerilog(new OutMod(2)))

Elaborating design...
Done elaborating.
module OutMod(
  input        clock,
  input        reset,
  output [3:0] io_m
);
  assign io_m = 4'h2; // @[cmd46.sc 7:10]
endmodule



defined class Mag
defined class OutMod

## `Bundle` Composition

* Bundles can ...
  * be _extended_
  * be nested
  * go in `Vec`s or include `Vec`s


In [48]:
class Mag extends Bundle {
    val m = Output(UInt(4.W))
}

class SignMag extends Mag {
    val s = Output(Bool())
}

class PairSignMag extends Bundle {
    val nums = Vec(2, new SignMag)
}

class OutMod(a: Int, b: Int) extends Module {
    val io = IO(new PairSignMag)
    io.nums(0).m := a.U
    io.nums(0).s := false.B
    io.nums(1).m := b.U
    io.nums(1).s := false.B
}

defined class Mag
defined class SignMag
defined class PairSignMag
defined class OutMod

In [48]:
//printVerilog(new OutMod(3,4))
// OK
//println(getVerilog(new CounterInstMod(4)))
// Unknow Error
//println(getVerilog(new OutMod(3, 4)))

## `cloneType` Boilerplate

* Unfortunately, due to embedding Chisel in Scala, had to include `cloneType`
  * Most often occurs when parameterizing a Bundle
* ~~Future versions of Chisel hope to have smoother work around~~
    * **Chisel 3.5 fixed this!**

In [49]:
class SignMag(n: Int) extends Bundle {
    val x = Output(UInt(n.W))
    val s = Output(Bool())
    // No longer necessary
    // override def cloneType = (new SignMag(n)).asInstanceOf[this.type]
}

class OutMod(n: Int, a: Int) extends Module {
    val io = IO(Output(new SignMag(8)))
    io.x := a.U
    io.s := false.B
}

defined class SignMag
defined class OutMod

In [49]:
//printVerilog(new OutMod(8,4))
// Unknow error
//println(getVerilog(new OutMod(8,4)))

## Working With Bundles Hierarchically

* `<>` is a _bulk connection_
    * Will connect an entire Bundle
* Newest versions of Chisel (newer than this course) have much [richer connection operators](https://www.chisel-lang.org/docs/explanations/connectable) available

In [50]:
class SignMag(w: Int) extends Bundle {
    val m = UInt(w.W)
    val s = Bool()
}

class PassThru(w: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(new SignMag(w))
        val out = Output(new SignMag(w))
    })
//     io.out.m := io.in.m
//     io.out.s := io.in.s
   io.in <> io.out
}

//printVerilog(new PassThru(4))

defined class SignMag
defined class PassThru

In [50]:
//printVerilog(new PassThru(4))
// Unknown error
//println(getVerilog(new PassThru(4)))

## Bundles Can Have Wires in Both Directions

* `Flipped` reverses directions



In [51]:
class Handshake(w: Int) extends Bundle {
    val ready = Input(Bool())
    val data  = Output(UInt(w.W))
}

class PassThru(w: Int) extends Module {
    val io = IO(new Bundle {
        val in = Flipped(new Handshake(w))
        val out = new Handshake(w)
    })
    io.in <> io.out
}

//printVerilog(new PassThru(4))

defined class Handshake
defined class PassThru

In [51]:
//printVerilog(new PassThru(4))
// Unknown error
//println(getVerilog(new PassThru(4)))

## Scala `Option`

* Scala's `Option[T]` is a type wrapper around `T` to indicate of potential non-existence
  * Is either `None` or `Some(x)`
* Brief Option API primer (will learn more graceful methods later)
  * `isDefined` - returns `Boolean` indicating if it is has something
  * `get` - returns value if it has something, otherwise exception
* _Motivation:_ can use Option to have optional fields in a Bundle

In [57]:
val o: Option[Int] = Some(4)
// val o: Option[Int] = None
if (o.isDefined)
    println("o.get:" + o.get)
else
    println("empty")

o.get:4


o: Option[Int] = Some(4)

## Making Optional IOs in Chisel

In [53]:
class MaybePair(w: Int, hasY: Boolean) extends Bundle {
    val x = Output(UInt(w.W))
    val y: Option[UInt] = if (hasY) Some(Output(UInt(w.W))) else None
}

class OutMod(w: Int, a: Int, useY: Boolean) extends Module {
    val io = IO(Output(new MaybePair(w, useY)))
    io.x := a.U
    if (useY)
//     if (io.y.isDefined)
        io.y.get := a.U
}

//printVerilog(new OutMod(8,4,true))

defined class MaybePair
defined class OutMod

In [53]:
//printVerilog(new OutMod(8,4,true))
// Unknown error
//println(getVerilog(new OutMod(8,4,true)))

## Scala `tabulate`

* May have seen this method used in assignments
* More general way (than `fill`) to populate a collection
* Will produce new collection by calling _anonymous function_ on every element
  * This function takes a single argument (index)
  * Can use `_` to wildcard replace first use, but be careful

In [54]:
Seq.fill(4)(0)
Seq.tabulate(4)(i => i)
Seq.tabulate(4)(_*2)

res53_0: Seq[Int] = List(0, 0, 0, 0)
res53_1: Seq[Int] = List(0, 1, 2, 3)
res53_2: Seq[Int] = List(0, 2, 4, 6)

## Summary

* We covered many mechanisms today - try to think of how to use them in your code to make it _safer_, more _productive_, or _easier to read_
  * Think about complexity, the best way to encapsulate it, both in hardware & software
* Methods are a great way to group hardware smaller than a module
  * Also allow for programmatic construction of hardware
* Factory methods (via companion object) - can be nice for overloading constructors
* Bundles are extremely versatile (not just I/O)